In [ ]:
"""
Automatic speech recognition and speaker identity
"""
from __future__ import division, print_function, absolute_import
import os
import numpy as np
import sklearn.preprocessing as prep
import librosa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D,LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses

In [2]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    '''
    classdocs
    '''
    def __init__(self, filePath="/home/francisco/voz"):
        '''
        Constructor
        '''
        basepath=filePath
        self.mfccs=[]
        self.speakers=[]
        self.words=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n))
            self.mfccs.append(mfccs)
            #decode file name format speaker_word_number
            speaker,word,number=n.split("_")
            self.speakers.append(speaker)
            self.words.append(word)
        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.leWords=prep.LabelEncoder()
        self.leWords.fit(self.words)
        self.iWordLabels=self.leWords.transform(self.words)
        print(len(self.words))
        print(self.words)
        print(self.speakers)

ea=EasyASR()
print(ea.leWords)

Imports doned
fNames= 108
16000 wave= (3871,) mfcc= (8, 20) file= /home/francisco/voz/paco_no_006.wav
16000 wave= (4963,) mfcc= (10, 20) file= /home/francisco/voz/paco_si_008.wav
16000 wave= (9285,) mfcc= (19, 20) file= /home/francisco/voz/paco_dos_002.wav
16000 wave= (4434,) mfcc= (9, 20) file= /home/francisco/voz/paco_uno_004.wav
16000 wave= (2428,) mfcc= (5, 20) file= /home/francisco/voz/paco_no_007.wav
16000 wave= (4882,) mfcc= (10, 20) file= /home/francisco/voz/paco_para_003.wav
16000 wave= (6972,) mfcc= (14, 20) file= /home/francisco/voz/paco_cuatro_003.wav
16000 wave= (6524,) mfcc= (13, 20) file= /home/francisco/voz/paco_nueve_001.wav
16000 wave= (4630,) mfcc= (10, 20) file= /home/francisco/voz/carlos_si_002.wav
16000 wave= (14450,) mfcc= (29, 20) file= /home/francisco/voz/paco_buenos-dias_001.wav
16000 wave= (14816,) mfcc= (29, 20) file= /home/francisco/voz/paco_buenos-dias_003.wav
16000 wave= (6847,) mfcc= (14, 20) file= /home/francisco/voz/irene_hola_003.wav
16000 wave= (9176

16000 wave= (5938,) mfcc= (12, 20) file= /home/francisco/voz/paco_casa_005.wav
108
['no', 'si', 'dos', 'uno', 'no', 'para', 'cuatro', 'nueve', 'si', 'buenos-dias', 'buenos-dias', 'hola', 'seis', 'buenos-dias', 'ocho', 'cinco', 'hola', 'hola', 'nueve', 'no', 'hola', 'si', 'hola', 'avanza', 'siete', 'cuatro', 'si', 'para', 'si', 'si', 'si', 'seis', 'buenos-dias', 'cinco', 'ocho', 'no', 'dos', 'buenos-dias', 'cinco', 'uno', 'nueve', 'si', 'no', 'casa', 'hola', 'no', 'retrocede', 'si', 'si', 'hola', 'tres', 'seis', 'ocho', 'avanza', 'cinco', 'no', 'hola', 'tres', 'retrocede', 'uno', 'si', 'avanza', 'uno', 'tres', 'dos', 'si', 'cuatro', 'cuatro', 'hola', 'no', 'casa', 'cinco', 'no', 'retrocede', 'siete', 'seis', 'cinco', 'hola', 'buenos-dias', 'para', 'buenos-dias', 'buenos-dias', 'si', 'dos', 'uno', 'siete', 'seis', 'si', 'casa', 'no', 'siete', 'buenos-dias', 'hola', 'siete', 'hola', 'tres', 'buenos-dias', 'casa', 'ocho', 'no', 'no', 'nueve', 'casa', 'no', 'buenos-dias', 'siete', 'nueve', 

In [3]:
trainX, trainYwords, trainYspeakers = np.array(ea.mfccs),np.array(ea.iWordLabels),np.array(ea.iSpeakerLabels)
#testX, testY = test

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputWords_dim=max(trainYwords)+1
outputSpeakers_dim=max(trainYspeakers)+1

print(outputWords_dim,outputSpeakers_dim)

# Converting labels to binary vectors
trainYwords_cat    = utils.to_categorical(trainYwords   ,outputWords_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
print (trainX.shape,trainYwords_cat.shape,trainYspeakers_cat.shape)

17 4
(108, 30, 20) (108, 17) (108, 4)


In [ ]:
# Network building
def build_model_asr_lstm(input_dim,outputWords_dim,outputSpeakers_dim,hiden_dim=128):
    input = Input(shape=(30,input_dim))
    #x = LSTM(hiden_dim, input_length=maxlen, input_dim=input_dim)(input)
    x = Bidirectional(LSTM(hiden_dim))(input)
    #x = Bidirectional(LSTM(hiden_dim,return_sequences=True))(input)
    #x = Dropout(0.5)(x)
    x = Bidirectional(LSTM(hiden_dim))(x)
    x = Dropout(0.5)(x)
    wd= Dense(hiden_dim)(x)
    sd= Dense(hiden_dim)(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_words    = Dense(outputWords_dim   , activation="softmax", name="word")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)

    model = Model(input, [prediction_words,prediction_speakers])
    return model

In [ ]:
model=build_model_asr_lstm(input_dim,outputWords_dim,outputSpeakers_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [7]:
# Training
history=model.fit(trainX, [trainYwords_cat,trainYspeakers_cat], batch_size=108,nb_epoch=50,validation_split=0.2)

Train on 86 samples, validate on 22 samples
Epoch 1/50
86/86 [==============================] - 6s 68ms/sample - loss: 4.4427 - word_loss: 2.9613 - speaker_loss: 1.4815 - word_accuracy: 0.0349 - speaker_accuracy: 0.2326 - val_loss: 3.4342 - val_word_loss: 2.6930 - val_speaker_loss: 0.7412 - val_word_accuracy: 0.0455 - val_speaker_accuracy: 0.8182
Epoch 2/50
86/86 [==============================] - 0s 315us/sample - loss: 3.6250 - word_loss: 2.7562 - speaker_loss: 0.8688 - word_accuracy: 0.1163 - speaker_accuracy: 0.8023 - val_loss: 3.0912 - val_word_loss: 2.5840 - val_speaker_loss: 0.5073 - val_word_accuracy: 0.2273 - val_speaker_accuracy: 0.8182
Epoch 3/50
86/86 [==============================] - 0s 322us/sample - loss: 3.1539 - word_loss: 2.5761 - speaker_loss: 0.5778 - word_accuracy: 0.2442 - speaker_accuracy: 0.8023 - val_loss: 2.9908 - val_word_loss: 2.4817 - val_speaker_loss: 0.5091 - val_word_accuracy: 0.2273 - val_speaker_accuracy: 0.8182
Epoch 4/50
86/86 [=====================

Epoch 28/50
86/86 [==============================] - 0s 289us/sample - loss: 0.0728 - word_loss: 0.0500 - speaker_loss: 0.0228 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 0.2836 - val_word_loss: 0.1542 - val_speaker_loss: 0.1294 - val_word_accuracy: 1.0000 - val_speaker_accuracy: 0.9091
Epoch 29/50
86/86 [==============================] - 0s 275us/sample - loss: 0.0595 - word_loss: 0.0452 - speaker_loss: 0.0143 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 0.2640 - val_word_loss: 0.1382 - val_speaker_loss: 0.1258 - val_word_accuracy: 1.0000 - val_speaker_accuracy: 0.9091
Epoch 30/50
86/86 [==============================] - 0s 333us/sample - loss: 0.0493 - word_loss: 0.0331 - speaker_loss: 0.0162 - word_accuracy: 1.0000 - speaker_accuracy: 1.0000 - val_loss: 0.2463 - val_word_loss: 0.1240 - val_speaker_loss: 0.1224 - val_word_accuracy: 1.0000 - val_speaker_accuracy: 0.9091
Epoch 31/50
86/86 [==============================] - 0s 347us/sample - loss: 0.

In [8]:
print(trainX[5:7,:,:].shape)
predWord,predSpeaker=model.predict(trainX[5:6,:,:])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

(2, 30, 20)
[[6.9077441e-04 3.6773039e-07 2.8914087e-06 1.8668650e-05 1.3194502e-06
  6.5095919e-06 2.1680782e-04 2.5589763e-05 1.9420349e-06 1.5823221e-05
  9.9882799e-01 2.1837368e-06 3.7675275e-06 1.1820406e-04 2.8638642e-06
  2.6985623e-07 6.4024920e-05]] [[1.4509758e-05 5.6985191e-06 3.4389617e-05 9.9994540e-01]]
10 = 0.998828
3 = 0.9999454
paco  ha dicho ' para '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']


In [1]:
!sox -r 16000 -t alsa default recording.wav silence 1 0.1 1% 1 1.5 1%
!aplay "recording.wav"


Input File     : 'default' (alsa)
Channels       : 2
Sample Rate    : 16000
Precision      : 16-bit
Sample Encoding: 16-bit Signed Integer PCM

In:0.00% 00:00:02.56 [00:00:00.00] Out:4.27k [      |      ]        Clip:0    
Done.
Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [23]:
!aplay "recording.wav"

Playing WAVE 'recording.wav' : Signed 16 bit Little Endian, Rate 16000 Hz, Stereo


In [12]:
sample=wav2mfcc("recording.wav")
sample=np.expand_dims(np.array(sample),axis=0)
sample.shape

16000 wave= (4268,) mfcc= (9, 20) file= recording.wav


(1, 30, 20)

In [13]:
predWord,predSpeaker=model.predict([sample])
print(predWord,predSpeaker)
y_predWord   = np.argmax(predWord)
y_predSpeaker= np.argmax(predSpeaker)
print(y_predWord,"=",np.max(predWord))
print(y_predSpeaker,"=",np.max(predSpeaker))
word=ea.leWords.inverse_transform([y_predWord])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
print(speaker[0]," ha dicho \'",word[0],"\'")
print(ea.leWords.classes_)
print(ea.leSpeakers.classes_)

[[5.3734693e-06 5.0096655e-06 4.8653033e-04 1.9779230e-07 4.5692109e-06
  4.6217865e-05 9.9916232e-01 2.1973645e-05 5.2174738e-07 5.1471884e-06
  2.4262894e-04 2.2538798e-07 1.2201465e-06 2.4372120e-08 1.1250569e-07
  1.7694147e-05 4.1547412e-07]] [[1.2177546e-05 2.5807108e-06 1.2578537e-05 9.9997258e-01]]
6 = 0.9991623
3 = 0.9999726
paco  ha dicho ' hola '
['avanza' 'buenos-dias' 'casa' 'cinco' 'cuatro' 'dos' 'hola' 'no' 'nueve'
 'ocho' 'para' 'retrocede' 'seis' 'si' 'siete' 'tres' 'uno']
['carlos' 'eva' 'irene' 'paco']
